In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_session_id = '137-5565607-0668155'
param_ubid_main = '135-9546146-6073418'
param_session_token = '3iuUPMVP9pez3dDPQbNug4OUnEGlaHZeQY9Lce8uZBCsPt4SKBcnO64MyZyzMa2RnnwQ1uME9HxpBWntNnbtQzSwlsoNuCEiRDccVzgX92EuNgQ3/11zpOlsqJdSYsBORGoLYBF62oKjF9owTY858bDLhii5koaRu+QGJ8WJUudg+Oaa3LDFmWWcmKNoUy8eYXv58AFaGQwBIx38AHJXAS8G5Jso/PCkh6HPeWeqBi1EKov9b1Mh/pST5ekzars+FQlSa58bO9iDhOSy4v9htikWD2im1H0XywbNJQNRFwoHGkM2Y5fWjV7B8CBcaWcHQcCQUNh5WUnWX9C0w5dZ4UGvdLucCbUC'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./2.menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests

        import sys
        sys.path.append('../00.Tools')
        from crawler_configuration import get_header, get_proxy

        from bs4 import BeautifulSoup
        from lxml import etree

        import re

        import json

        if input_.loc[a, 'Market'] == 'US':
            url = f'''https://www.amazon.com/dp/{input_.loc[a, 'ASIN']}?language=en_US'''

        cookie = {'session-id': param_session_id,
                  'ubid-main': param_ubid_main,
                  'session-token': param_session_token}

        try:
            while True:
                
                b = 0
                while True:

                    while True:
                        b += 1
                        try:
                            resp = requests.get(url,
                                                headers=get_header(),
                                                proxies=get_proxy(),
                                                cookies=cookie,
                                                timeout=(10, 10))
                            break
                        except:
                            continue

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        # = = = = = = = = = = = = = = =

                        list_captcha = html.xpath('//div[@class="a-box-inner"]/h4/text()')

                        # = = = = = = = = = = = = = = =

                        if not list_captcha or list_captcha[0].strip() != 'Enter the characters you see below':
                            break

                # = = = = = = = = = = = = = = =

                zip_code = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', html.xpath('//span[@id="glow-ingress-line2"]/text()')[0]).strip()

                # = = = = = = = = = = = = = = =

                price = html.xpath('//span[@class="a-offscreen"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                json_ = json.loads('{"initial":[{' + resp.text.split('\'colorImages\': { \'initial\': [{')[1].split('}]},')[0] + '}]}')

                # = = = = = = = = = = = = = = =

                dict_src = {}
                list_src = [src['hiRes'].strip() if src['hiRes'] else src['large'].strip() for src in json_['initial']]
                for i, src in zip(range(len(list_src)), list_src):
                    dict_src[str(i)] = src

                # = = = = = = = = = = = = = = =

                dict_overview = {}
                list_tr = html.xpath('//table[@class="a-normal a-spacing-micro"]/tr')
                for i, tr in zip(range(len(list_tr)), list_tr):
                    list_span = [re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', text).strip() for text in tr.xpath('./td/span/text()')]
                    dict_overview[str(i)] = {list_span[0]: list_span[1]}

                # = = = = = = = = = = = = = = =

                about = ''
                list_span = [re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', text).strip() for text in html.xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]/li/span/text()')]
                for i, span in zip(range(len(list_span)), list_span):
                    about += f'{i+1}. {span}\n'

                # = = = = = = = = = = = = = = =

                json_ = json.loads(resp.text.split('jQuery.parseJSON(\'')[1].split('}\');')[0] + '}')

                # = = = = = = = = = = = = = = =

                title = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', json_['title']).strip()

                # = = = = = = = = = = = = = = =

                parent_asin = json_['parentAsin'].strip()

                # = = = = = = = = = = = = = = =

                dict_detail = {}
                list_tr = html.xpath('//table[@id="productDetails_techSpec_section_1"]/tr')
                for i, tr in zip(range(len(list_tr)), list_tr):
                    dict_detail[str(i)] = {re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip(): re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}

                # = = = = = = = = = = = = = = =

                dict_information = {}
                list_tr = html.xpath('//table[@id="productDetails_detailBullets_sections1"]/tr')
                for i, tr in zip(range(len(list_tr)), list_tr):
                    th = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
                    if th == 'Customer Reviews':
                        dict_information[str(i)] = {th: {}}
                        dict_information[str(i)][th]['0'] = tr.xpath('./td/div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()
                        dict_information[str(i)][th]['1'] = tr.xpath('./td/div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()
                    elif th == 'Best Sellers Rank':
                        dict_information[str(i)] = {th: {}}
                        list_span = [etree.tostring(span).decode('utf-8') for span in tr.xpath('./td/span/span')]
                        list_span = [re.sub(r'<.*?>|\(.*?\)', '', span).strip() for span in list_span]
                        list_href = [f'https://www.amazon.com{href.strip()}' for href in tr.xpath('./td/span/span/a/@href')]
                        for j, span, href in zip(range(len(list_span)), list_span, list_href):
                            dict_information[str(i)][th][str(j)] = {span: href}
                    else:
                        dict_information[str(i)] = {th: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}

                # = = = = = = = = = = = = = = =

                dict_detail_2 = {}
                list_span = html.xpath('//div[@id="detailBulletsWrapper_feature_div"]/div[@id="detailBullets_feature_div"]/ul/li/span')
                for i, span in zip(range(len(list_span)), list_span):
                    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', span.xpath('./span/text()')[0]).strip()
                    dict_detail_2[str(i)] = {key[:-1].strip() if key.endswith(':') else key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', span.xpath('./span/text()')[1]).strip()}

                # = = = = = = = = = = = = = = =

                dict_information_2 = {}
                list_span = html.xpath('//div[@id="detailBulletsWrapper_feature_div"]/ul[@class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"]/li/span')
                for i, span in zip(range(len(list_span)), list_span):
                    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', span.xpath('./span/text()')[0]).strip()
                    key = key[:-1].strip() if key.endswith(':') else key
                    if key == 'Customer Reviews':
                        dict_information_2[str(i)] = {key: {}}
                        dict_information_2[str(i)][key]['0'] = span.xpath('./div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()
                        dict_information_2[str(i)][key]['1'] = span.xpath('./div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()
                    elif key == 'Best Sellers Rank':
                        dict_information_2[str(i)] = {key: {}}
                        list_span_2 = [etree.tostring(span_2).decode('utf-8') for span_2 in span.xpath('./descendant-or-self::span[@class="a-list-item"]')]
                        list_span_2 = [re.sub(r'<ul.*?</ul>|<span.*?Best Sellers Rank.*?</span>|<.*?>|\(.*?\)', '', span_2).strip() for span_2 in list_span_2]
                        list_href = [f'https://www.amazon.com{href.strip()}' for href in span.xpath('./descendant-or-self::a/@href')]
                        for j, span_2, href in zip(range(len(list_span_2)), list_span_2, list_href):
                            dict_information_2[str(i)][key][str(j)] = {span_2: href}
                    else:
                        raise

                # = = = = = = = = = = = = = = =

                if not dict_detail and not dict_detail_2 and not dict_information and not dict_information_2:
                    continue

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'ASIN': input_.loc[a, 'ASIN'],
                                         'Parent ASIN': parent_asin,
                                         'Title': title,
                                         'ZIP Code': zip_code,
                                         'Price': price,
                                         'About': about.strip(),
                                         'Picture': '',
                                         'Url': url,
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Overview': json.dumps(dict_overview),
                                         'Json_Detail 1': json.dumps(dict_detail),
                                         'Json_Detail 2': json.dumps(dict_detail_2),
                                         'Json_Information 1': json.dumps(dict_information),
                                         'Json_Information 2': json.dumps(dict_information_2)}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                break
        except:
            df_temp = pd.DataFrame([{'Market': input_.loc[a, 'Market'],
                                     'Page': input_.loc[a, 'Page'],
                                     'Page Url': input_.loc[a, 'Page Url'],
                                     'No': input_.loc[a, 'No'],
                                     'ASIN': input_.loc[a, 'ASIN']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'ASIN']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./3.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：100

[状态：ok，尝试次数：1] - 2.B009B5S5E8
[剩余数量：88] - [当前时间：19:24:36]

[状态：ok，尝试次数：1] - 3.B0CN8J4K53
[剩余数量：87] - [当前时间：19:24:36]

[状态：ok，尝试次数：18] - 7.B0CYYYL59H
[剩余数量：86] - [当前时间：19:24:37]

[状态：ok，尝试次数：23] - 1.B08W98H7H7
[剩余数量：85] - [当前时间：19:24:38]

[状态：ok，尝试次数：9] - 15.B00OU7PZCM
[剩余数量：84] - [当前时间：19:24:42]

[状态：ok，尝试次数：12] - 10.B09PH28FG2
[剩余数量：83] - [当前时间：19:24:48]

[状态：ok，尝试次数：42] - 8.B009TKMXV6
[剩余数量：82] - [当前时间：19:24:49]

[状态：ok，尝试次数：112] - 6.B08TTC1HQ3
[剩余数量：81] - [当前时间：19:24:49]

[状态：ok，尝试次数：29] - 11.B0BF9SR4VN
[剩余数量：80] - [当前时间：19:24:50]

[状态：ok，尝试次数：48] - 9.B08W9MX59Y
[剩余数量：79] - [当前时间：19:24:52]

[状态：ok，尝试次数：36] - 5.B0CW2SKD6R
[剩余数量：78] - [当前时间：19:24:53]

[状态：ok，尝试次数：116] - 14.B0BPCP4XQN
[剩余数量：77] - [当前时间：19:24:55]

[状态：ok，尝试次数：91] - 17.B0BY213GX3
[剩余数量：76] - [当前时间：19:24:58]

[状态：ok，尝试次数：49] - 18.B01KHL31PW
[剩余数量：75] - [当前时间：19:25:00]

[状态：ok，尝试次数：22] - 22.B097JWWR4V
[剩余数量：74] - [当前时间：19:25:00]

[状态：ok，尝试次数：19] - 19.B09V88967X
[剩余数量：73] - [当前时间：19:25:01]

[状态：ok，尝试次数：47] - 13.B00